## Query Processing

functions are defined to tackle the 4 types of operations: AND, OR, AND NOT, OR NOT.
The index and doc IDs are loaded from their pickle files.
Queries are processed from left to right.  

**Input Format**    

The first line contains the number of queries, N. \
The next 2N lines would represent the queries. \
Each query would consist of two lines: \
(a) line 1: Input sentence \
(b) line 2: Input operation sequence

**Output Format**  

Number of documents matched: Returned value \
No. of comparisons required: Returned value


In [74]:
#importing neccesary files

import nltk
import re
from pprint import pprint
import pickle as pkl
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from ipynb.fs.defs.data_preprocessing import remove_punc, remove_art_connector, remove_emoji
from nltk.corpus import stopwords
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frostrot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading Data


In [75]:
# Loading Pickle files

with open("./pickle_files/index.pkl","rb") as f:
    posting = pkl.load(f)
    
with open("./pickle_files/doc_id.pkl","rb") as f:
    doc_id = pkl.load(f)

### Filtering

In [76]:
#Remove stopwords, and convert shorted words into there extended forms

def stopword(text):
  EXTENDED_FORMS = {"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'i would', "i'll": 'i will', "i'm": 'i am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'i have', "let's": 'let us', "mightn't": 'might not', "mustn't": 'must not',"n't": 'not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who've": 'who have', "won't": 'will not', "wouldn't": 'would not', "you'd": 'you would', "you'll": 'you will', "you're": 'you are', "you've": 'you have', "'re": ' are', "wasn't": 'was not', "we'll": 'we will', "'cause": 'because', "could've": 'could have', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd've": 'i would have', "i'll've": 'i will have', "it'd": 'it would', "it'd've": 'it would have', "it'll've": 'it will have', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't've": 'might not have', "must've": 'must have', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "sha'n't": 'shall not', "shan't've": 'shall not have', "she'd've": 'she would have', "she'll've": 'she will have', "should've": 'should have', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so as', "this's": 'this is', "that'd": 'that would', "that'd've": 'that would have', "there'd": 'there would', "there'd've": 'there would have', "here's": 'here is', "they'd've": 'they would have', "they'll've": 'they will have', "to've": 'to have', "we'd've": 'we would have', "we'll've": 'we will have', "what'll've": 'what will have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where've": 'where have', "who'll've": 'who will have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't've": 'will not have', "would've": 'would have', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have', "you'd've": 'you would have', "you'll've": 'you will have'}
  x= word_tokenize(text)
  for i in range(len(x)):
    if x[i] in EXTENDED_FORMS:
      x[i] = EXTENDED_FORMS[x[i]]
    if x[i] in stopwords.words('english'):
      x[i]=''
  x=remove_punc(x)
  x=remove_art_connector(x)
  return " ".join(x)

In [77]:
def stemming(text):
    x = text.split(" ")
    ps = PorterStemmer()
    return " ".join([ps.stem(i) for i in x])

In [78]:
#Filter the parsed text, by, converting them into lowercase, removing any tags, extra spaces.

def filter(item):
  if type(item)==str:
    item=item.lower()
    item=re.sub('[#@]\w+\s*',"",item)
    item=re.sub(r'\\N','',item)
    item=remove_emoji(item)
    item=stopword(item)
    item=stemming(item)
  return item

### Operator Functions

In [79]:
#Merging Code, for two documents (basically merging two sorted list)
# OR function

def func_OR(document_word1, document_word2):    
    n,m = len(document_word1), len(document_word2)
    i,j = 0,0
    comparision = 0
    merged_documents = []

    while i<n and j<m:
        comparision+=1
        if document_word1[i] < document_word2[j]:
            merged_documents.append(document_word1[i])
            i+=1
        elif document_word1[i]> document_word2[j]:
            merged_documents.append(document_word2[j])
            j+=1
        else:
            merged_documents.append(document_word1[i])
            i+=1
            j+=1
    
    while i<n:
        merged_documents.append(document_word1[i])
        i+=1
    
    while j<m:
        merged_documents.append(document_word2[j])
        j+=1

    return merged_documents, comparision

In [80]:
#And functions, comparing and merging if the words are present in both the documents or not. (check if a number present in a sorted list)

def func_AND(document_word1, document_word2):
    n,m = len(document_word1), len(document_word2)
    i,j = 0,0
    comparision = 0
    merged_documents = []

    while i<n and j<m:
        comparision+=1
        if document_word1[i] == document_word2[j]:
            merged_documents.append(document_word1[i])
            i+=1
            j+=1
        elif document_word1[i] < document_word2[j]:
            i+=1
        else:
            j+=1

    return merged_documents, comparision

In [81]:
#Function to calculation ornot, first all documents with no, word2, present in it, and then performs the or functions.

def func_ORNOT(document_word1, document_word2):
    
    i,j = 0,document_word2[0]
    n,m = len(document_word1), len(document_word2)
    comparision = 0

    negated_document_word2 = [x for x in range(j)]

    while j < document_word2[-1]:
        comparision += 1
        
        if i<m and document_word2[i]==j:
            i+=1
        else:
            negated_document_word2.append(j)
        j+=1 

    negated_document_word2.extend([x for x in range(document_word2[-1]+1,len(doc_id))])
    
    i,j = 0,0
    merged_documents = []
    comparision = 0
    n,m = len(document_word1), len(negated_document_word2)
    while i<n and j<m:
        comparision+=1
        if document_word1[i] < negated_document_word2[j]:
            merged_documents.append(document_word1[i])
            i+=1
        elif document_word1[i]> negated_document_word2[j]:
            merged_documents.append(negated_document_word2[j])
            j+=1
        else:
            merged_documents.append(document_word1[i])
            i+=1
            j+=1
    
    while i<n:
        merged_documents.append(document_word1[i])
        i+=1
    
    while j<m:
        merged_documents.append(negated_document_word2[j])
        j+=1

    return merged_documents, comparision

In [82]:
#Function to calculate and not, where a word should be in document1 but not in document 2.

def func_ANDNOT(document_word1, document_word2):
    
    n,m = len(document_word1), len(document_word2)
    i,j = 0,0
    comparision = 0
    merged_documents = []

    while i<n and j<m:
        comparision+=1
        if document_word1[i]<document_word2[j]:
            merged_documents.append(document_word1[i])
            i+=1
        elif document_word1[i]== document_word2[j]:
            i+=1
            j+=1
        else:
            j+=1
            
    while i < n:
        merged_documents.append(document_word1[i])
        i+=1

    return merged_documents, comparision

### Processing

In [83]:
#processing the queries.

def processing(words, operations):
    if len(words)==0 or len(operators)==0:
        if len(words)!=0 and words[0] in posting:
            return posting[words[0]], 0
        else:
            return [], 0

    words_doc_mapping = []

    for word in words:
        if word in posting:
            words_doc_mapping.append(posting[word])
        else:
            words_doc_mapping.append([])
    
    documents = words_doc_mapping[0]
    total_comparison = 0

    for i,operation in enumerate(operations):
        documents, comparisons = operation(documents,words_doc_mapping[i+1])
        total_comparison+=comparisons

    return documents, total_comparison

### Quering Inputs

In [84]:
# Queries Input

maps = {'OR': func_OR,'AND': func_AND,'OR NOT': func_ORNOT,'AND NOT': func_ANDNOT}

N = int(input("Enter number of queries: "))

for n in range(N):
    sentence = input("Enter sentence: ").strip()
    sentence_words = filter(sentence).split()
    print("Sentence Words to be processed after filtering: ",sentence_words)
    operators = list(map(str,input("Enter operators; comma seperated: ").split(',')))
    operation = []

    for operator in operators:
        if operator.upper() in maps:
            operation.append(maps[operator.upper()])

    print("Operations to be done on the Sentence: ",operators)

    if len(sentence_words)-len(operators) == 1:    
        documents, total_comparisons = processing(sentence_words, operation)
        print("Number of documents retrieved: ", len(documents))
        print("Number of comparisons made: ", total_comparisons)
        print("Document Names:\n",[doc_id[i] for i in documents])
        print()
    else:
        print("Error")

Sentence Words to be processed after filtering:  ['lion', 'stood', 'thought', 'moment']
Operations to be done on the Sentence:  ['or not', 'or', 'and']
Number of documents retrieved:  152
Number of comparisons made:  3218
Document Names:
 ['a_tv_t-p.com', 'aeonint.txt', 'allusion', 'ambrose.bie', 'anim_lif.txt', 'anime.lif', 'annoy.fascist', 'art-fart.hum', 'b-2.jok', 'badday.hum', 'barney.txt', 'bbh_intv.txt', 'beauty.tm', 'beesherb.txt', 'bitnet.txt', 'butwrong.hum', 'bw-phwan.hat', 'bw.txt', 'caesardr.sal', 'candy.txt', 'cartoon.law', 'cartoon.laws', 'cartoon_.txt', 'chickenheadbbs.txt', 'clancy.txt', 'classicm.hum', 'cmu.share', 'collected_quotes.txt', 'commutin.jok', 'conan.txt', 'consp.txt', 'cookie.1', 'coyote.txt', 'cybrtrsh.txt', 'dead3.txt', 'dead4.txt', 'dead5.txt', 'dingding.hum', 'doggun.sto', 'drinks.gui', 'econridl.fun', 'english.txt', 'episimp2.txt', 'exam.50', 'facedeth.txt', 'fascist.txt', 'female.jok', 'filmgoof.txt', 'flux_fix.txt', 'fuckyou2.txt', 'gas.txt', 'gd_ql